<h1>Sparse Convolutional Denoising Autoencoders for Genotype Imputation <span class="tocSkip"></span></h1>

### ORIGINAL CODE FROM https://github.com/work-hard-play-harder/SCDA. Just change versiones of libraries or functions names wich have changed

<h1>Table of Contents<span class="tocSkip"></span></h1>
<div class="toc"><ul class="toc-item"><li><span><a href="#Introduction" data-toc-modified-id="Introduction-1"><span class="toc-item-num">1&nbsp;&nbsp;</span>Introduction</a></span></li><li><span><a href="#Dataset" data-toc-modified-id="Dataset-2"><span class="toc-item-num">2&nbsp;&nbsp;</span>Dataset</a></span><ul class="toc-item"><li><span><a href="#Loading-data" data-toc-modified-id="Loading-data-2.1"><span class="toc-item-num">2.1&nbsp;&nbsp;</span>Loading data</a></span></li><li><span><a href="#Preprocessing" data-toc-modified-id="Preprocessing-2.2"><span class="toc-item-num">2.2&nbsp;&nbsp;</span>Preprocessing</a></span></li></ul></li><li><span><a href="#Method" data-toc-modified-id="Method-3"><span class="toc-item-num">3&nbsp;&nbsp;</span>Method</a></span><ul class="toc-item"><li><span><a href="#Load-model" data-toc-modified-id="Load-model-3.1"><span class="toc-item-num">3.1&nbsp;&nbsp;</span>Load model</a></span></li><li><span><a href="#Prediction-on-test-data" data-toc-modified-id="Prediction-on-test-data-3.2"><span class="toc-item-num">3.2&nbsp;&nbsp;</span>Prediction on test data</a></span></li></ul></li></ul></div>

# Introduction

This notebook demonstrates a case study of testing a SCDA model on yeast genotype dataset with 10% missing genotypes. 

In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split

from tensorflow import keras
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, UpSampling1D, Dropout
from tensorflow.keras.regularizers import l1
from tensorflow.keras.utils import to_categorical

from tensorflow.keras.models import load_model

# Dataset

## Loading data

In [2]:
input_name = '../data/yeast/yeast_genotype_test.txt'
df_ori = pd.read_csv(input_name, sep='\t', index_col=0)
df_ori.shape

(877, 28220)

In [3]:
df_ori.head()

,33070_chrI_33070_A_T,33147_chrI_33147_G_T,33152_chrI_33152_T_C,33200_chrI_33200_C_T,33293_chrI_33293_A_T,33328_chrI_33328_C_A,33348_chrI_33348_G_C,33403_chrI_33403_C_T,33502_chrI_33502_A_G,33548_chrI_33548_A_C,...,12048853_chrXVI_925593_G_C,12049199_chrXVI_925939_T_C,12049441_chrXVI_926181_C_T,12050613_chrXVI_927353_T_G,12051167_chrXVI_927907_A_C,12051240_chrXVI_927980_A_G,12051367_chrXVI_928107_C_T,12052782_chrXVI_929522_C_T,12052988_chrXVI_929728_A_G,12053130_chrXVI_929870_C_T
SAMID,,,,,,,,,,,,,,,,,,,,,
40_73,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2
40_74,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
40_75,2,2,2,2,2,2,2,2,2,2,...,1,1,1,1,1,1,1,1,1,1
40_76,2,2,2,2,2,2,2,2,2,2,...,2,2,2,2,2,2,2,2,2,2
40_77,1,1,1,1,1,1,1,1,1,1,...,2,2,2,2,2,2,2,2,2,2


## Preprocessing

In [4]:
# one hot encode
test_X = to_categorical(df_ori)
test_X.shape

(877, 28220, 3)

# Method

## Load model

In [10]:
# returns a compiled model
SCDA = load_model('../models/SCDA_yeast.keras')

## Prediction on test data

In [11]:
# hyperparameters
missing_perc = 0.1

In [12]:
test_X_missing = test_X.copy()
test_X_missing.shape

(877, 28220, 3)

In [13]:
def cal_prob(predict_missing_onehot):
    # calcaulate the probility of genotype 0, 1, 2
    predict_prob = predict_missing_onehot[:,:,1:3] / predict_missing_onehot[:,:,1:3].sum(axis=2, keepdims=True)
    return predict_prob[0]

In [17]:
avg_accuracy = []
for i in range(test_X_missing.shape[0]):
    # Generates missing genotypes
    missing_size = int(missing_perc * test_X_missing.shape[1])
    missing_index = np.random.randint(test_X_missing.shape[1],
                                      size=missing_size)
    test_X_missing[i, missing_index, :] = [1, 0, 0]

    # predict
    predict_onehot = SCDA.predict(test_X_missing[i:i + 1, :, :])
    # only care the missing position
    predict_missing_onehot = predict_onehot[0:1, missing_index, :]
    
    # calculate probability and save file.
    predict_prob = cal_prob(predict_missing_onehot)
    pd.DataFrame(predict_prob).to_csv('../results/{}SCDAyeast.csv'.format(df_ori.index[i]),
                                      header=[1, 2],
                                      index=False)

    # predict label
    predict_missing = np.argmax(predict_missing_onehot, axis=2)
    # real label
    label_missing_onehot = test_X[i:i + 1, missing_index, :]
    label_missing = np.argmax(label_missing_onehot, axis=2)
    # accuracy
    correct_prediction = np.equal(predict_missing, label_missing)
    accuracy = np.mean(correct_prediction)
    print('{}/{}, sample ID: {}, accuracy: {:.4f}'.format(
        i, test_X_missing.shape[0], df_ori.index[i], accuracy))

    avg_accuracy.append(accuracy)

1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
0/877, sample ID: 40_73, accuracy: 0.9965
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
1/877, sample ID: 40_74, accuracy: 0.9989
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
2/877, sample ID: 40_75, accuracy: 0.9982
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
3/877, sample ID: 40_76, accuracy: 0.9996
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
4/877, sample ID: 40_77, accuracy: 0.9986
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
5/877, sample ID: 40_79, accuracy: 0.9982
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
6/877, sample ID: 40_80, accuracy: 0.9979
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 21ms/step
7/877, sample ID: 40_81, accuracy: 0.9989
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
8/877, sample ID: 40_82, accuracy: 0.9986
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
9/877, sample ID: 40_83, accuracy: 0.9993
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step
10/877, sample ID: 40_84, accuracy: 0.9986
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 19ms/step
11/877, sample ID: 40_85, accuracy: 0.9975
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 18ms/step


In [18]:
print('The average imputation accuracy' \
      'on test data with {} missing genotypes is {:.4f}: '
    .format(missing_perc, np.mean(avg_accuracy)))

The average imputation accuracyon test data with 0.1 missing genotypes is 0.9979: 
